<a href="https://colab.research.google.com/github/Rainniee/TextSummarization_PointerGenerator/blob/master/Chatbot%20Modeling_Seq2SeqTF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
dataset = pd.read_csv("drive/My Drive/Assignment 4 Chatbot/articles_abstract_pairs/article-abstract-tokenized.csv")

In [0]:
from keras.models import Model
from keras.layers import Embedding, Dense, Input, RepeatVector, concatenate, Dropout
from keras.layers.recurrent import LSTM
from keras.preprocessing.sequence import pad_sequences
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.python.ops.rnn import dynamic_rnn
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

Using TensorFlow backend.


In [0]:
RNN_SIZE = 128
BATCH_SIZE = 64
EPOCHS = 3
KEEP_PROBABILITY = 0.5
OPTIMIZER_TYPE = 'adam'
LEARNING_RATE = 0.001
EMBEDDING_SIZE = 100

data_dir_path = 'drive/My Drive/Assignment 4 Chatbot'
data_file = '/articles_abstract_pairs/article-abstract-tokenized.csv'
glove_file = '/articles_abstract_pairs/glove.6B.' + str(EMBEDDING_SIZE) + 'd.txt'

In [0]:
VERBOSE = 1
NUM_LAYERS = 3
MAX_INPUT_SEQ_LENGTH = 500
MAX_TARGET_SEQ_LENGTH = 50
MAX_INPUT_VOCAB_SIZE = 3000
MAX_TARGET_VOCAB_SIZE = 1000
NUM_SAMPLES = 5000
MAX_DECODER_SEQ_LENGTH = 4

In [0]:
def def_keras_optimizer():
    if OPTIMIZER_TYPE == 'sgd':
        # default LEARNING_RATE = 0.01
        keras_optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, momentum=0.0, decay=0.0, nesterov=False)
    elif OPTIMIZER_TYPE == 'rmsprop':
        # default LEARNING_RATE = 0.001
        keras_optimizer = keras.optimizers.RMSprop(lr=LEARNING_RATE, rho=0.9, epsilon=None, decay=0.0)
    elif OPTIMIZER_TYPE == 'adagrad':
        # default LEARNING_RATE = 0.01
        keras_optimizer = keras.optimizers.Adagrad(lr=LEARNING_RATE, epsilon=None, decay=0.0)
    elif OPTIMIZER_TYPE == 'adadelta':
        # default LEARNING_RATE = 1.0
        keras_optimizer = keras.optimizers.Adadelta(lr=LEARNING_RATE, rho=0.95, epsilon=None, decay=0.0)
    else:   # OPTIMIZER_TYPE == 'adam':
        # default LEARNING_RATE = 0.001
        keras_optimizer = keras.optimizers.Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0,
                                                amsgrad=False)

    return keras_optimizer

In [0]:
def def_tf_optimizer(lr):
    if OPTIMIZER_TYPE == 'sgd':
        # default LEARNING_RATE = 0.01
        tf_optimizer = tf.train.GradientDescentOptimizer(lr)
    elif OPTIMIZER_TYPE == 'rmsprop':
        # default LEARNING_RATE = 0.001
        tf_optimizer = tf.train.RMSPropOptimizer(lr)
    elif OPTIMIZER_TYPE == 'adagrad':
        # default LEARNING_RATE = 0.01
        tf_optimizer = tf.train.AdagradOptimizer(lr)
    elif OPTIMIZER_TYPE == 'adadelta':
        # default LEARNING_RATE = 1.0
        tf_optimizer = tf.train.AdadeltaOptimizer(lr)
    else:   # OPTIMIZER_TYPE == 'adam':
        # default LEARNING_RATE = 0.001
        tf_optimizer = tf.train.AdamOptimizer(lr)

    return tf_optimizer

### Modeling

In [0]:
def preprocess_data(input_texts, target_texts):
    vocab = set()
    vocab.add('<PAD>')
    vocab.add('END')
    vocab.add('<UNK>')
    vocab.add('START')

    for input_text in input_texts:
        for word in input_text:
            if word not in vocab:
                vocab.add(word)
    for target_text in target_texts:
        for word in target_text:
            if word not in vocab:
                vocab.add(word)

    vocab = sorted(list(vocab))
    word2idx = dict([(word, i) for i, word in enumerate(vocab)])
    idx2word = dict((i, word) for word, i in word2idx.items())

    source_text_id = []
    target_text_id = []

    for i in range(len(input_texts)):
        source_sentence = input_texts[i]
        target_sentence = target_texts[i]

        source_token_id = []
        target_token_id = []

        for index, token in enumerate(source_sentence):
            if token != "":
                source_token_id.append(word2idx[token])

        for index, token in enumerate(target_sentence):
            if token != "":
                target_token_id.append(word2idx[token])

        target_token_id.append(word2idx['END'])

        source_text_id.append(source_token_id)
        target_text_id.append(target_token_id)

    config = dict()
    config['word2idx'] = word2idx
    config['idx2word'] = idx2word
    config['source_text_id'] = source_text_id
    config['target_text_id'] = target_text_id

    return config

In [0]:
def get_accuracy(target, logits):
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0, 0), (0, max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0, 0), (0, max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

In [0]:
class TfSeq2Seq(object):

    def __init__(self, config):
        self.word2idx = config['word2idx']
        self.idx2word = config['idx2word']
        self.source_text_id = config['source_text_id']
        self.target_text_id = config['target_text_id']

    def encoding_layer(self, input_data, keep_prob):
        embed = tf.contrib.layers.embed_sequence(input_data,
                                                 vocab_size=len(self.word2idx),
                                                 embed_dim=EMBEDDING_SIZE)

        stacked_cells = tf.contrib.rnn.MultiRNNCell(
            [tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.LSTMCell(RNN_SIZE), keep_prob) for _ in
             range(NUM_LAYERS)])

        outputs, state = dynamic_rnn(stacked_cells, embed, dtype=tf.float32)
        return outputs, state

    def process_decoder_input(self, targets):
        go_id = self.word2idx['START']

        after_slice = tf.strided_slice(targets, [0, 0], [BATCH_SIZE, -1], [1, 1])
        after_concat = tf.concat([tf.fill([BATCH_SIZE, 1], go_id), after_slice], 1)

        return after_concat

    def decoding_layer(self, dec_input, encoder_state, keep_prob, target_sequence_length, max_target_len):
        target_vocab_size = len(self.word2idx)
        dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, EMBEDDING_SIZE]))
        dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

        cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.LSTMCell(RNN_SIZE) for _ in range(NUM_LAYERS)])

        with tf.variable_scope("decode"):
            output_layer = tf.layers.Dense(target_vocab_size)
            dec_cell_train = tf.contrib.rnn.DropoutWrapper(cells, output_keep_prob=keep_prob)

            helper_train = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_sequence_length)

            decoder_train = tf.contrib.seq2seq.BasicDecoder(dec_cell_train, helper_train, encoder_state,
                                                            output_layer)

            train_output, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder_train, impute_finished=True,
                                                                   maximum_iterations=max_target_len)

        with tf.variable_scope("decode", reuse=True):
            start_sequence_id = self.word2idx['START']
            end_sequence_id = self.word2idx['END']

            dec_cell_infer = tf.contrib.rnn.DropoutWrapper(cells, output_keep_prob=keep_prob)

            helper_infer = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, tf.fill([BATCH_SIZE],
                                                                                            start_sequence_id),
                                                                    end_sequence_id)

            decoder_infer = tf.contrib.seq2seq.BasicDecoder(dec_cell_infer, helper_infer, encoder_state, output_layer)

            infer_output, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder_infer, impute_finished=True,
                                                                   maximum_iterations=max_target_len)

        return train_output, infer_output

    def seq2seq_model(self, input_data, targets, keep_prob, target_sequence_length, max_target_len):

        enc_outputs, enc_states = self.encoding_layer(input_data, keep_prob)

        dec_input = self.process_decoder_input(targets)

        train_output, infer_output = self.decoding_layer(dec_input, enc_states, keep_prob, target_sequence_length,
                                                         max_target_len)

        return train_output, infer_output

    def get_batches(self, sources, targets):
        for batch_i in range(0, len(sources) // BATCH_SIZE):
            start_i = batch_i * BATCH_SIZE

            sources_batch = sources[start_i:start_i + BATCH_SIZE]
            targets_batch = targets[start_i:start_i + BATCH_SIZE]

            pad_int = self.word2idx['<PAD>']
            max_sentence_source = max([len(sentence) for sentence in sources_batch])
            max_sentence_target = max([len(sentence) for sentence in targets_batch])
            pad_sources_batch = np.array([sentence + [pad_int] * (max_sentence_source - len(sentence)) for sentence in
                                          sources_batch])
            pad_targets_batch = np.array([sentence + [pad_int] * (max_sentence_target - len(sentence)) for sentence in
                                          targets_batch])

            pad_targets_lengths = []
            for target in pad_targets_batch:
                pad_targets_lengths.append(len(target))

            pad_source_lengths = []
            for source in pad_sources_batch:
                pad_source_lengths.append(len(source))

            yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths

    def fit(self):
        train_graph = tf.Graph()
        with train_graph.as_default():
            inputs = tf.placeholder(tf.int32, [None, None], name='input')
            input_data = tf.reverse(inputs, [-1])
            targets = tf.placeholder(tf.int32, [None, None], name='targets')
            target_sequence_length = tf.placeholder(tf.int32, [None], name='target_sequence_length')
            max_target_len = tf.reduce_max(target_sequence_length)
            lr_rate = tf.placeholder(tf.float32, name='lr_rate')
            keep_prob = tf.placeholder(tf.float32, name='keep_prob')
            train_logits, inference_logits = self.seq2seq_model(input_data, targets, keep_prob, target_sequence_length,
                                                                max_target_len)
            training_logits = tf.identity(train_logits.rnn_output, name='logits')
            inference_logits = tf.identity(inference_logits.sample_id, name='predictions')
            masks = tf.sequence_mask(target_sequence_length, max_target_len, dtype=tf.float32, name='masks')

            with tf.name_scope("optimization"):

                cost = tf.contrib.seq2seq.sequence_loss(training_logits, targets, masks)
                # optimizer = tf.train.AdamOptimizer(lr_rate)
                optimizer = def_tf_optimizer(lr_rate)
                gradients = optimizer.compute_gradients(cost)
                capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not
                                    None]
                train_op = optimizer.apply_gradients(capped_gradients)

            train_source = self.source_text_id[BATCH_SIZE:]
            train_target = self.target_text_id[BATCH_SIZE:]
            valid_source = self.source_text_id[:BATCH_SIZE]
            valid_target = self.target_text_id[:BATCH_SIZE]

            (valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths) = \
                next(self.get_batches(valid_source, valid_target))

            sess = tf.Session(graph=train_graph)
            sess.run(tf.global_variables_initializer())

            for epoch_i in range(EPOCHS):
                for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                        self.get_batches(train_source, train_target)):
                    _, loss = sess.run([train_op, cost], {inputs: source_batch, targets: target_batch,
                                                          lr_rate: LEARNING_RATE,
                                                          target_sequence_length: targets_lengths,
                                                          keep_prob: KEEP_PROBABILITY})

                    if batch_i > 0:
                        if batch_i % 5 == 0:
                            batch_train_logits = sess.run(
                             inference_logits, {input_data: source_batch, target_sequence_length: targets_lengths,
                                                keep_prob: 1.0})

                            batch_valid_logits = sess.run(
                                inference_logits, {inputs: valid_sources_batch,
                                                   target_sequence_length:  valid_targets_lengths,
                                                   keep_prob: 1.0})

                            train_acc = get_accuracy(target_batch, batch_train_logits)
                            valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                            print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}'
                                  ', Loss: {:>6.4f}'.format(epoch_i, batch_i, len(self.source_text_id) // BATCH_SIZE,
                                                            train_acc, valid_acc, loss))
        return target_sequence_length, keep_prob, inputs, sess, inference_logits

    def summarize(self, inputs, text, sess, inference_logits, target_sequence_length, keep_prob):
        translate_sentence = []
        for word in text:
            if word in self.word2idx:
                translate_sentence.append(self.word2idx[word])
            else:
                translate_sentence.append(self.word2idx['<UNK>'])

        translate_logits = sess.run(inference_logits, {inputs: [translate_sentence] * BATCH_SIZE,
                                                       target_sequence_length: [len(
                                                           translate_sentence) * 2] * BATCH_SIZE,
                                                       keep_prob: 1.0})[0]

        return " ".join([self.idx2word[i] for i in translate_logits])


In [0]:
def main_tf_seq2seq():
    train = pd.read_csv(data_dir_path + data_file)
    resumes = []
    articles = []
    for resume in train[train.columns[0]].values:
        resumes.append(resume.split(' '))
    for article in train[train.columns[1]].values:
        articles.append(article.split(' '))

    config = preprocess_data(articles[:NUM_SAMPLES], resumes[:NUM_SAMPLES])
    summarizer = TfSeq2Seq(config)
    target_sequence_length, keep_prob, inputs, sess, inference_logits = summarizer.fit()
    for i in np.random.permutation(np.arange(len(articles)))[0:20]:
        x = articles[i]
        abstract = summarizer.summarize(inputs, x, sess, inference_logits, target_sequence_length, keep_prob)
        print('Article: ', articles)
        print('Generated Abstract: ', abstract)
        print('Original Article: ', x)

In [0]:
main_tf_seq2seq()

Epoch   0 Batch    5/78 - Train Accuracy: 0.2982, Validation Accuracy: 0.2982, Loss: 9.9277
Epoch   0 Batch   10/78 - Train Accuracy: 0.2244, Validation Accuracy: 0.2982, Loss: 9.4792
Epoch   0 Batch   15/78 - Train Accuracy: 0.3112, Validation Accuracy: 0.2982, Loss: 8.5717
Epoch   0 Batch   20/78 - Train Accuracy: 0.3546, Validation Accuracy: 0.2982, Loss: 7.6394
Epoch   0 Batch   25/78 - Train Accuracy: 0.3817, Validation Accuracy: 0.2982, Loss: 7.0327
Epoch   0 Batch   30/78 - Train Accuracy: 0.4336, Validation Accuracy: 0.2982, Loss: 6.2012
Epoch   0 Batch   35/78 - Train Accuracy: 0.2060, Validation Accuracy: 0.2982, Loss: 7.2832
Epoch   0 Batch   40/78 - Train Accuracy: 0.3086, Validation Accuracy: 0.2982, Loss: 6.4660
Epoch   0 Batch   45/78 - Train Accuracy: 0.2330, Validation Accuracy: 0.2982, Loss: 7.0080
Epoch   0 Batch   50/78 - Train Accuracy: 0.2372, Validation Accuracy: 0.2982, Loss: 7.0505
Epoch   0 Batch   55/78 - Train Accuracy: 0.2656, Validation Accuracy: 0.2982, L

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

